In [1]:
import lammps_logfile
import scipy
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg, NavigationToolbar2Tk
import PySimpleGUI as sg
import numpy as np
import PySimpleGUI as sg
import matplotlib.widgets as wg
import matplotlib
import matplotlib.pyplot as plt

rdf_dict = {}

In [2]:
#third version, using log_lammps library
def read_log_file(file):
    units=[]
    log=lammps_logfile.File(file)
    file.seek(0)
    for line in file:
        if 'units' in line:
            line=line.partition(' ')[2]
            line=line.rstrip()
            units.append(line)
        if 'Unit style' in line:
            line=line.partition(': ')[2]
            line=line.rstrip()
            units.append(line)
    return log, units

In [3]:
def read_RDF_file(file):

    x = []
    for line in file: 
        if not line.startswith("#"):
            t = line.strip('\n').split()
            t = [float(x) for x in t]
            if len(t) == 2:
                #print(t)
                key, N = str(int(t[0])), t[1]

            else:
                x.append(t)
                
            if len(x) == N:
                #rdf_dict[key] = x
                rdf_dict.update({key : x})
                x = []
    return rdf_dict

In [4]:
def linearFit(x,y,x_unit,y_unit,lower_bound_id=0,higher_bound_id=0):
    
    if higher_bound_id!=0 and higher_bound_id>lower_bound_id:
        if higher_bound_id>x.size:
            higher_bound_id=x.size
        x_new=x[lower_bound_id:higher_bound_id]
        y_new=y[lower_bound_id:higher_bound_id]
    else:
        x_new=x
        y_new=y
    a,b,c,d,e=scipy.stats.linregress(x_new,y_new)
    
    a_unit=y_unit+'/'+x_unit
    
    return a,b,a_unit

In [5]:
units_dict={'metal':{'Step':'','Time':'ps','TotEng':'eV','Temp':'K','Volume':'A^3','Density':'g/cm^3','Press':'bar','Ndanger':'','PotEng':'eV','KinEng':'eV','Enthalpy':'eV'},
      'real':{'Step':'','Time':'fs','TotEng':'kcal/mol','Temp':'K','Volume':'A^3','Density':'g/cm^3','Press':'atm','Ndanger':'','PotEng':'kcal/mol','KinEng':'kcal/mol','Enthalpy':'kcal/mol'},
      'SI':{'Step':'','Time':'s','TotEng':'J','Temp':'K','Volume':'m^3','Density':'kg/m^3','Press':'Pa','Ndanger':'','PotEng':'J','KinEng':'J','Enthalpy':'J'},
      'LJ':{'Step':'','Time':'','TotEng':'','Temp':'','Volume':'','Density':'','Press':'','Ndanger':'','PotEng':'','KinEng':'','Enthalpy':''}}

def units(units_type,x):
    return units_dict[units_type][x]

In [6]:
log_dict={}

In [7]:
def draw_figure_w_toolbar(canvas, fig, canvas_toolbar):
    if canvas.children:
        for child in canvas.winfo_children():
            child.destroy()
    if canvas_toolbar.children:
        for child in canvas_toolbar.winfo_children():
            child.destroy()
    figure_canvas_agg = FigureCanvasTkAgg(fig, master=canvas)
    figure_canvas_agg.draw()
    toolbar = Toolbar(figure_canvas_agg, canvas_toolbar)
    toolbar.update()
    figure_canvas_agg.get_tk_widget().pack(side='right', fill='both', expand=1)

In [8]:
class Toolbar(NavigationToolbar2Tk):
    def __init__(self, *args, **kwargs):
        super(Toolbar, self).__init__(*args, **kwargs)

In [9]:
def make_fit_window():
    fit_headings=['color','a','b','lim1','lim2']
    table=sg.Table(values=[], headings=fit_headings,
                            #max_col_width=65,
                            #col_widths=data_cols_width,
                            #auto_size_columns=True,
                            #justification='left',
                            enable_events=True,
                            enable_click_events=True,
                            expand_x=True,
                            select_mode=sg.TABLE_SELECT_MODE_BROWSE,
                            #num_rows=6,
                            key='table')
    
    layout=[
        [sg.T('N:'), sg.Combo([],key='combo_nfit',expand_x=True),
            sg.T('X:'), sg.Combo([],key='combo_xfit',expand_x=True),
            sg.T('Y:'), sg.Combo([],key='combo_yfit',expand_x=True),
            sg.B('Plot',key='Plotfit')],
        [sg.Canvas(key='controls_cv_fit')],
        [sg.Canvas(key='fig_cv_fit',
                       # it's important that you set this size
                       size=(600 * 2, 600), expand_x=True,expand_y=True
                       )],
        [sg.T("Fit parameters: (Intersection point will be calculated for two latest selected lines)")],
        [sg.Column(
            layout=[[table]],
            pad=(0, 0),
            expand_x=True, expand_y=True),
         sg.Column(
            layout=[[sg.B('Intersection point')], [sg.Input(key = 'interception', disabled = True, use_readonly_for_disable = True)]],
            pad=(0, 0))]
                  

    ]
    
    return sg.Window("Fit Window",layout,finalize=True,resizable=True)

In [10]:
def make_rdf_window():   
    layout=[
        [sg.FilesBrowse('Load RDF Files...',enable_events=True, key="file_rdf")],
        [sg.T('N:'), sg.Combo([],key='combo_nrdf',expand_x=True),
            sg.T('X:'), sg.Combo([],key='combo_xrdf',expand_x=True),
            sg.T('Y:'), sg.Combo([],key='combo_yrdf',expand_x=True),
        sg.B('Plot',key='Plotrdf')],
        [sg.Canvas(key='controls_cv_rdf')],
        [sg.Canvas(key='fig_cv_rdf',
                       # it's important that you set this size
                       size=(600 * 2, 600), expand_x=True,expand_y=True
                       )]
    ]
    return sg.Window("RDF Window",layout,finalize=True,resizable=True)

In [11]:
def make_main_window():
    layout = [
    [sg.FilesBrowse('Load Log Files...',enable_events=True, key="file_main")],
    [sg.TabGroup([[]], key = 'tab_group_main')],
    [sg.B('Plot'),sg.B('Linear Fit'), sg.B('RDF'), sg.B('Exit')],
    [sg.Canvas(key='controls_cv')],
    [sg.Column(
        layout=[
            [sg.T('N:'), sg.Combo([],key='combo_n1',expand_x=True),
            sg.T('X:'), sg.Combo([],key='combo_x1',expand_x=True),
            sg.T('Y:'), sg.Combo([],key='combo_y1',expand_x=True),
            sg.T('N:'), sg.Combo([],key='combo_n2',expand_x=True),
            sg.T('X:'), sg.Combo([],key='combo_x2',expand_x=True),
            sg.T('Y:'), sg.Combo([],key='combo_y2',expand_x=True)
            ],
            [sg.Canvas(key='fig_cv',
                       # it's important that you set this size
                       size=(600 * 2, 600), expand_x=True,expand_y=True
                       )],
            [sg.T('N:'), sg.Combo([],key='combo_n3',expand_x=True),
            sg.T('X:'), sg.Combo([],key='combo_x3',expand_x=True),
            sg.T('Y:'), sg.Combo([],key='combo_y3',expand_x=True),
            sg.T('N:'), sg.Combo([],key='combo_n4',expand_x=True),
            sg.T('X:'), sg.Combo([],key='combo_x4',expand_x=True),
            sg.T('Y:'), sg.Combo([],key='combo_y4',expand_x=True)
            ]
        ],
        #background_color='#DAE0E6',
        pad=(0, 0),
        expand_x=True, expand_y=True
    )]
    ]
    return sg.Window('MDGlass viewer', layout, resizable=True,finalize=True)

In [12]:
def intersection(a1,b1,a2,b2):
    x_intersection = (b2 - b1) / (a1 - a2)
    y_intersection = a1 * x_intersection + b1
    return x_intersection, y_intersection
    

In [13]:
def find_closest_point(x, y, x0, y0):
    min_distance = float('inf')
    closest_point_index = -1
    x_norm = abs(np.max(x)-np.min(x))
    y_norm = abs(np.max(y)-np.min(y))
    x00 = x0 / x_norm
    y00 = y0 / y_norm
    

    for i in range(len(x)):
        xp = x[i] / x_norm
        yp = y[i] / y_norm
        distance = np.sqrt((xp - x00)**2 + (yp - y00)**2)

        if distance < min_distance:
            min_distance = distance
            closest_point_index = i

    x_res = x[closest_point_index]
    y_res = y[closest_point_index]

    return x_res, y_res

In [14]:
def find_nearest_value(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx]

In [15]:
def plot_main(log, unit_types, fig_cv,toolbar_cv, values):
    n, xq, yq, ut, x, y = [], [], [], [], [], []
    #do zmiany
    for i in range(4):
        n.append(values['combo_n' + str(i + 1)] - 1)
        xq.append(values['combo_x' + str(i + 1)])
        yq.append(values['combo_y' + str(i + 1)])
        ut.append(unit_types[n[i]])
        x.append(log.get(xq[i], n[i]))
        y.append(log.get(yq[i], n[i]))
            
    fig,axs=plt.subplots(2, 2)
    a=lambda x: 1 if x > 1 else 0
    b=lambda x: 0 if x % 2 == 0 else 1
    for i in range(4):
        axs[a(i), b(i)].plot(x[i], y[i])
        axs[a(i), b(i)].set(xlabel = xq[i] + " [" + units(ut[i], xq[i]) + "]",
                            ylabel = yq[i] + " [" + units(ut[i], yq[i]) + "]")

    fig.tight_layout()
    
    plt.figure(1)
    fig = plt.gcf()
    plt.close()
    DPI = fig.get_dpi()
    #you have to play with this size to reduce the movement error
    #when the mouse hovers over the figure, it's close to canvas size
    fig.set_size_inches(500 * 2 / float(DPI), 500 / float(DPI))
    draw_figure_w_toolbar(fig_cv, fig, toolbar_cv)

In [16]:
def plot_rdf(log, unit_types, fig_cv,toolbar_cv, xq, yq, n):
    ut = unit_types[n]
    x = log.get(xq, n)
    y = log.get(yq, n)
    ux = units(ut, xq)
    uy = units(ut, yq)
    
    #x_rdf=[]
    #y_rdf=[]
            
    fig,axs=plt.subplots(1, 2)
    axs[0].scatter(x, y)
    axs[0].set(xlabel = xq + " [" + ux + "]",
                  ylabel = yq + " [" + uy + "]")
    
    #plot rdf
    #axs[1].plot(x_rdf, y_rdf)
    axs[1].set(xlabel = "Distance [A]", ylabel = "RDF")

    fig.tight_layout()
    
    plt.figure(1)
    fig = plt.gcf()
    plt.close()
    DPI = fig.get_dpi()
    #you have to play with this size to reduce the movement error
    #when the mouse hovers over the figure, it's close to canvas size
    fig.set_size_inches(500 * 2 / float(DPI), 500 / float(DPI))
    draw_figure_w_toolbar(fig_cv, fig, toolbar_cv)
    
    return fig, axs[0], axs[1], x, y, ux, uy

In [17]:
def plot_fit(log, unit_types, fig_cv,toolbar_cv, values):
    n = values['combo_nfit'] - 1
    xq = values['combo_xfit']
    yq = values['combo_yfit']
    ut = unit_types[n]
    x = log.get(xq, n)
    y = log.get(yq, n)
    ux = units(ut, xq)
    uy = units(ut, yq)
            
    fig,axs=plt.subplots(1)
    axs.plot(x, y)
    axs.set(xlabel = xq + " [" + ux + "]",
                  ylabel = yq + " [" + uy + "]")

    fig.tight_layout()
    
    plt.figure(1)
    fig = plt.gcf()
    plt.close()
    DPI = fig.get_dpi()
    #you have to play with this size to reduce the movement error
    #when the mouse hovers over the figure, it's close to canvas size
    fig.set_size_inches(500 * 2 / float(DPI), 500 / float(DPI))
    draw_figure_w_toolbar(fig_cv, fig, toolbar_cv)
    
    return fig, axs, x, y, ux, uy